# Time Series Analysis and Forecasting

## Overview
This notebook provides a comprehensive guide to time series analysis and forecasting techniques using Python.

## Contents
1. Data Loading and Preprocessing
2. Exploratory Data Analysis
3. Time Series Decomposition
4. Stationarity Testing
5. Model Building (ARIMA, Prophet)
6. Forecasting and Evaluation

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Data Loading and Preprocessing

In [ ]:
# Load sample data (replace with your actual data)
# For demonstration, we'll create synthetic time series data
np.random.seed(42)
dates = pd.date_range(start='2020-01-01', end='2023-12-31', freq='D')
trend = np.linspace(100, 200, len(dates))
seasonal = 20 * np.sin(2 * np.pi * np.arange(len(dates)) / 365.25)
noise = np.random.normal(0, 10, len(dates))
values = trend + seasonal + noise

df = pd.DataFrame({
    'date': dates,
    'value': values
})

df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

print("Data shape:", df.shape)
print("Data info:")
print(df.info())
print("\nFirst few rows:")
print(df.head())

## 2. Exploratory Data Analysis

In [ ]:
# Basic statistics
print("Basic Statistics:")
print(df.describe())

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

In [ ]:
# Time series plot
plt.figure(figsize=(15, 6))
plt.plot(df.index, df['value'])
plt.title('Time Series Data')
plt.xlabel('Date')
plt.ylabel('Value')
plt.grid(True)
plt.show()

## 3. Time Series Decomposition

In [ ]:
# Perform seasonal decomposition
decomposition = seasonal_decompose(df['value'], model='additive', period=365)

fig, axes = plt.subplots(4, 1, figsize=(15, 12))

df['value'].plot(ax=axes[0], title='Original')
decomposition.trend.plot(ax=axes[1], title='Trend')
decomposition.seasonal.plot(ax=axes[2], title='Seasonal')
decomposition.resid.plot(ax=axes[3], title='Residual')

plt.tight_layout()
plt.show()

## 4. Stationarity Testing

In [ ]:
# Augmented Dickey-Fuller test
def adf_test(series):
    result = adfuller(series)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value}')
    
    if result[1] <= 0.05:
        print("The series is stationary")
    else:
        print("The series is non-stationary")

print("ADF Test Results:")
adf_test(df['value'])

## 5. Model Building

### ARIMA Model

In [ ]:
# Split data
train_size = int(len(df) * 0.8)
train, test = df['value'][:train_size], df['value'][train_size:]

# Fit ARIMA model
arima_model = ARIMA(train, order=(1, 1, 1))
arima_fit = arima_model.fit()

# Forecast
forecast = arima_fit.forecast(steps=len(test))

# Evaluate
arima_mae = mean_absolute_error(test, forecast)
arima_rmse = np.sqrt(mean_squared_error(test, forecast))

print(f"ARIMA MAE: {arima_mae:.4f}")
print(f"ARIMA RMSE: {arima_rmse:.4f}")

# Plot results
plt.figure(figsize=(15, 6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index, forecast, label='ARIMA Forecast')
plt.legend()
plt.title('ARIMA Model Results')
plt.show()

### Prophet Model

In [ ]:
# Prepare data for Prophet
prophet_df = df.reset_index().rename(columns={'date': 'ds', 'value': 'y'})

# Split data
train_prophet = prophet_df[:train_size]
test_prophet = prophet_df[train_size:]

# Fit Prophet model
prophet_model = Prophet()
prophet_model.fit(train_prophet)

# Make predictions
future = prophet_model.make_future_dataframe(periods=len(test_prophet))
forecast = prophet_model.predict(future)

# Evaluate
prophet_forecast = forecast['yhat'][-len(test_prophet):]
prophet_mae = mean_absolute_error(test_prophet['y'], prophet_forecast)
prophet_rmse = np.sqrt(mean_squared_error(test_prophet['y'], prophet_forecast))

print(f"Prophet MAE: {prophet_mae:.4f}")
print(f"Prophet RMSE: {prophet_rmse:.4f}")

# Plot results
plt.figure(figsize=(15, 6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index, prophet_forecast, label='Prophet Forecast')
plt.legend()
plt.title('Prophet Model Results')
plt.show()

## 6. Model Comparison

In [ ]:
# Compare models
results = pd.DataFrame({
    'Model': ['ARIMA', 'Prophet'],
    'MAE': [arima_mae, prophet_mae],
    'RMSE': [arima_rmse, prophet_rmse]
})

print("Model Comparison:")
print(results)

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

results.plot(x='Model', y='MAE', kind='bar', ax=axes[0])
axes[0].set_title('MAE Comparison')
axes[0].set_ylabel('MAE')

results.plot(x='Model', y='RMSE', kind='bar', ax=axes[1])
axes[1].set_title('RMSE Comparison')
axes[1].set_ylabel('RMSE')

plt.tight_layout()
plt.show()

## Conclusion

This notebook demonstrates various time series analysis techniques including:
- Data preprocessing and exploration
- Time series decomposition
- Stationarity testing
- ARIMA and Prophet model building
- Model evaluation and comparison

Feel free to replace the synthetic data with your actual time series data!